<a href="https://colab.research.google.com/github/MikolajKasprzyk/metal_archives_statistics/blob/main/metal_archives_processing_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import pickle
import string
import time
import numpy as np
import re
from datetime import datetime

In [ ]:
path = '/content/drive/My Drive/Colab Notebooks/metal_archives/'

path_out = ('/content/drive/My Drive/Colab Notebooks/metal_archives/'
            'visualization_files/')

bands_info_raw = pd.read_pickle(path + 'bands_info_df.pkl')

### Cleaning data

In [ ]:
# DISCOGRAPHY
bands_info_clean = bands_info_raw.copy()

def discog_cleaning(albums):
    # check if discog value is nan
    if not isinstance(albums, list):
        return np.nan
    # delete empty lists
    else:    
        albums = [album for album in albums if album]
        # splitting last value (number of reviews and avg rating in %) into
        # two ints loop doesnt change instances in place, so make a copy
        # and override if there are no records in discography
        # we get one string, chage it to np.nan
        if len(albums[0]) > 1:
            albums_ = []
            for album in albums:
                if album[-1]: # if there is rating
                    rating = album[-1].split()[-1].strip('()').split('%')[0]
                    revs_num = album[-1].split(' ')[0]
                    album = album[:-1] + [int(rating)] + [int(revs_num)]
                else: # if there are no reviews add 0 revies and nan rating
                    album = album[:-1] + [0] + [np.nan]
                albums_.append(album)
            albums = albums_
        else:
            albums = np.nan
        return albums

bands_info_clean.loc[:, 'discog'] = bands_info_raw['discog']\
                                    .apply(lambda x: discog_cleaning(x))

In [ ]:
# GENRE
# split string to list, dlete word metal, delete empty strings
def genre_to_list(text):
    if isinstance(text, str):  # Check if text is a string
        # delete word 'Metal' unless it is 'Post-Metal'
        text = text.replace('Post-Metal', 'Postmetal')
        # do not want to split 
        pattern = r'Metal\b|\)|\(|early|later|mid|,|;|with|elements|'\
                 'influences|(?<=\w)-|-(?=\w)'
        text = re.sub(pattern, '', text)
        text = text.replace('Crust Punk', 'Crust-Punk')\
            .replace('Hardcore Punk', 'HC-Punk')\
            .replace('Hard ', 'Hard-' ).replace('Melodic ', 'Melodic-')\
            .replace(" 'n' ", "-n-")\
            .replace('Atmospheric ', 'Atmospheric-')\
            .replace('Post ', 'Post-')\
            .replace('Symphonic ', 'Symphonic-')\
            .replace('Progressive ', 'Progressive-')\
            .replace('Brutal ', 'Brutal-')\
            .replace('/', ' ')
        text_list = text.split(' ')
        # Delete empty strings
        text_list = [text for text in text_list if text]
        return text_list
    else:
        return []
        
bands_info_clean.loc[:,'genre'] = bands_info_raw['genre'].apply(
                                  lambda x: genre_to_list(x))

In [ ]:
# LYRICAL THEMES
# split string to list, delete empty strings
def lyrical_themes_to_list(text):
    if isinstance(text, str):  # Check if text is a string
        text = text.replace('N/A', '')
        text_list = text.split(',')
        # Delete empty strings and whitespaces
        text_list = [text for text in text_list if text]
        return text_list
    else:
        return []
        
bands_info_clean.loc[:, 'lyrical_themes'] = bands_info_raw['lyrical_themes'].\
        apply(lambda x: lyrical_themes_to_list(x))

In [ ]:
# BANDS FORMATION AND SPLIT YEAR
# Replace 'present' with the current year
bands_info_clean['years_active'] = bands_info_clean['years_active'].str.replace(
                                   'present', str(datetime.now().year))

# Extract all years in the 'years_active' column
years_active = bands_info_clean['years_active'].str.findall(r'\b\d{4}\b')

# Convert 'formed_in' column to pd.Timestamp, if 'N/A' then pd.NaT
formed_in = bands_info_clean['formed_in'] = pd.to_datetime(
        bands_info_clean['formed_in'], format='%Y', errors='coerce')
# Create 'end_year' column by extracting the last year from 'years_active'
end_year_mask = years_active.apply(lambda x: len(x) > 0)
end_year = pd.to_datetime(years_active[end_year_mask].apply(lambda x: x[-1]), 
                          format='%Y', errors='coerce')
bands_info_clean.loc[end_year_mask, 'end_year'] = end_year
bands_info_clean.loc[~end_year_mask, 'end_year'] = pd.NaT

# Drop the 'years_active' column if no longer needed
bands_info_clean.drop('years_active', axis=1, inplace=True)

### Preparing tables for visualisation

In [ ]:
#COUNTRY - countries information, population for counting 
# bands and records per capita
country_raw = pd.read_csv(path + 'world_population.csv')

country = country_raw[['Country/Territory', 'Continent',
       '2020 Population','2010 Population', '2000 Population',
       '1990 Population', '1980 Population', '1970 Population']].copy()

country.columns = country.columns.str.replace(' Population', '_population')
country = country.rename(columns={'Country/Territory':'name',
        'Continent':'continent'}).rename_axis('country_id').add_prefix('country_')
# start indexing at 1 so I can use 0 for NaN values when country info is missing
country.index = country.index + 1

country.to_csv(path_out + 'country.csv')

In [ ]:
# BANDS - main dataframe with bands information
band = bands_info_clean.copy()
# rename columns for consistency
band = band.rename(columns={'status':'band_status',
                            'formed_in':'band_start_year',
                            'current_label':'band_label',
                            'end_year':'band_stop_year'})
# replacing country name with country_id - less redundant data
band['country_id'] = band['country_of_origin'].map(
                     country.reset_index().set_index('country_name')['id']
                     ).fillna(0).astype(int)
# drop other columns that will have separate dataframes connected by band_id
band = band.drop(columns=['country_of_origin', 'artists', 'discog', 'genre',
                          'lyrical_themes']).rename_axis('band_id')

band.to_csv(path_out + 'band.csv')

In [ ]:
band.head()

,band_url,band_name,band_status,band_start_year,band_label,band_stop_year,country_id
band_id,,,,,,,
0,https://www.metal-archives.com/bands/A_--_Solu...,A // Solution,Split-up,NaT,Voltic Records,NaT,222
1,https://www.metal-archives.com/bands/A_12_Gaug...,A 12 Gauge Tragedy,Split-up,2006-01-01,Unsigned/independent,2009-01-01,222
2,https://www.metal-archives.com/bands/A_Balance...,A Balance of Power,Active,2004-01-01,Unsigned/independent,2023-01-01,222
3,https://www.metal-archives.com/bands/A_Band_of...,A Band of Orcs,Active,2007-01-01,Grimpire Entertainment,2023-01-01,222
4,https://www.metal-archives.com/bands/A_Baptism...,A Baptism by Fire,Active,2015-01-01,Unsigned/independent,2023-01-01,36


In [ ]:
# RECORD - dataframe with all records information
# delete bands with no discography information and explode by discog column
records_df = bands_info_clean[
            bands_info_clean['discog'].apply(lambda x: isinstance(x, list))]\
            .explode('discog')

bands_info_clean = bands_info_clean[
                   bands_info_clean['discog'].apply(lambda x: isinstance(x, list))]
# create new columns from data of every record
new_cols = {'record_title': records_df['discog'].apply(lambda x: x[0]),
            'record_type': records_df['discog'].apply(lambda x: x[1]),
            'record_year': records_df['discog'].apply(lambda x: x[2]),
            'record_rating': records_df['discog'].apply(lambda x: x[3]),
            'record_rating_count': records_df['discog'].apply(lambda x: x[4])}

# replace coutry_of_origin column with country_id from country dataframe
records_df['country_id'] = records_df['country_of_origin'].map(
                     country.reset_index().set_index('country_name')['id']
                     ).fillna(0).astype(int)
# assign new columns and drop columns that are redundant
records_df = records_df.assign(**new_cols).drop(columns=['discog',
            'country_of_origin', 'band_url', 'artists', 'band_name', 'status',
            'formed_in', 'genre', 'lyrical_themes', 
            'current_label', 'end_year'])
records_df['record_rating_count'] = records_df['record_rating_count']\
                                    .fillna(0).astype(int)

record = records_df.reset_index(drop=False).rename(columns={'index':'band_id'})\
        .rename_axis('record_id')

record.to_csv(path_out + 'record.csv')

In [ ]:
# ACTIVE BANDS BY YEAR BY COUNTRY
# Copy the relevant columns into a new DataFrame
band_years_temp = bands_info_clean[['formed_in',
                                    'end_year',
                                    'country_of_origin']].copy()
band_years_temp.reset_index(inplace=True)

band_years_temp.rename(columns={'index':'band_id',
                                'country_of_origin':'country_name'},
                                inplace=True)
# Fill missing values in 'end_year' and 'formed_in' columns + - one year, as 
# quite a lot have missing data. Dropping NaN values would be another approach.
band_years_temp['end_year'].fillna(band_years_temp['formed_in'] 
                                + pd.offsets.DateOffset(years=1), inplace=True)
band_years_temp['formed_in'].fillna(band_years_temp['end_year'] 
                                - pd.offsets.DateOffset(years=1), inplace=True)

# Drop bands with missing all active years information
band_years_temp.dropna(subset=['formed_in'], inplace=True)

# Create an empty DataFrame to store the data
band_active_years = pd.DataFrame(
                    columns=['year', 'band_id', 'country_name']).copy()
# pandas date range from first date in bands_years_temp to present
years = pd.date_range(start=band_years_temp['formed_in'].min(),
                      end=pd.Timestamp.now(), freq='Y')
# Iterate over years and band_years_temp DataFrame
for year in years:
    mask = ((year >= band_years_temp['formed_in']) &
           (year <= band_years_temp['end_year']))
    active_bands = band_years_temp[mask].copy()
    active_bands['year'] = year
    band_active_years = pd.concat([band_active_years, active_bands[
                        ['year', 'band_id', 'country_name']]], axis=0)

band_active_years['year'] = band_active_years['year'].dt.strftime('%Y')
band_active_years.set_index('year', inplace=True)
# replace some names to be consistent with country dataframe
country_replacements = {'Korea, South': 'South Korea', 
                        'Czechia': 'Czech Republic', 'Türkiye': 'Turkey'}
band_active_years['country_name'] = band_active_years['country_name']\
                                    .replace(country_replacements)
# add country_id and drop country name - less redundant data
band_active_years['country_id'] = band_active_years['country_name'].map(
                    country.reset_index().set_index('country_name')['id']
                    ).fillna(0).astype(int)
band_active_years.drop(columns='country_name', inplace=True)

band_active_years.to_csv(path_out + 'band_active_years.csv')

In [ ]:
# GENRE
genre_df = bands_info_clean.copy().drop(columns=['band_url', 'artists',
            'discog', 'band_name', 'country_of_origin', 'status', 'formed_in', 
            'lyrical_themes', 'current_label', 'end_year'])\
            .rename(columns={'genre':'genre_name'})

genre_df = genre_df.explode('genre_name').rename_axis('band_id').dropna()
# create dataframe with unique genre_name and genre_id
genre = pd.DataFrame({'genre_name': genre_df['genre_name'].unique()})\
        .rename_axis('genre_id')
# ma genre_id and drop genre_name - less redundant data
genre_df['genre_id'] = genre_df['genre_name'].map(genre.reset_index()\
                        .set_index('genre_name')['genre_id'])

band_genre = genre_df.drop(columns='genre_name')

band_genre.to_csv(path_out + 'band_genre.csv')
genre.to_csv(path_out + 'genre.csv')

In [ ]:
# ARTISTS
artist_df = bands_info_clean.copy().drop(columns=['band_url', 'genre',
            'discog', 'band_name', 'country_of_origin', 'status', 'formed_in', 
            'lyrical_themes', 'current_label', 'end_year'])\
            .rename(columns={'artists':'artist_name'})
artist_df = artist_df.explode('artist_name').rename_axis('band_id').dropna()

artist = pd.DataFrame({'artist_name': artist_df['artist_name'].unique()})\
        .rename_axis('artist_id')

artist_df['artist_id'] = artist_df['artist_name'].map(artist.reset_index()\
                        .set_index('artist_name')['artist_id'])

band_artist = artist_df.drop(columns='artist_name')

artist.to_csv(path_out + 'artist.csv')
band_artist.to_csv(path_out + 'band_artist.csv')

In [ ]:
# LYRICS
lyrics_df = bands_info_clean.copy().drop(columns=['band_url', 'genre',
            'artists', 'discog', 'band_name', 'country_of_origin', 'status', 
            'formed_in', 'current_label', 'end_year'])\
            .rename(columns={'lyrical_themes':'lyrics_theme'})
lyrics_df = lyrics_df.explode('lyrics_theme').rename_axis('band_id').dropna()

lyrics = pd.DataFrame({'lyrics_theme': lyrics_df['lyrics_theme'].unique()})\
        .rename_axis('lyrics_id')

lyrics_df['lyrics_id'] = lyrics_df['lyrics_theme'].map(lyrics.reset_index()\
                        .set_index('lyrics_theme')['lyrics_id'])

band_lyrics = lyrics_df.drop(columns='lyrics_theme')

lyrics.to_csv(path_out + 'lyrics.csv')
band_lyrics.to_csv(path_out + 'band_lyrics.csv')